In [ ]:
import requests
import pandas as pd
import certifi
import io

# Your Census API Key
api_key = "34ddb1ab7d24edad01946182fca5446955cdc8aa"

# Census API endpoint for ACS 5-Year Data (2023)
base_url = "https://api.census.gov/data/2023/acs/acs5"

# Variables to fetch
variables = ["B25071_001E"]  # Median Gross Rent as % of Income
fields = ",".join(variables)

# Fetch data for all counties with certificate verification using certifi
params = {
    "get": f"{fields},NAME",
    "for": "county:*",
    "key": api_key
}
response = requests.get(base_url, params=params, verify=certifi.where())
data = response.json()

# Convert to DataFrame
columns = ["Median Rent % Income", "County", "State Code", "County Code"]
df = pd.DataFrame(data[1:], columns=columns)

# Convert Rent % Income to numeric
df["Median Rent % Income"] = pd.to_numeric(df["Median Rent % Income"], errors="coerce")

# Merge with state FIPS codes for easier mapping
state_fips_url = "https://www2.census.gov/geo/docs/reference/state.txt"
# Fetch the state FIPS data with proper SSL certificate verification
state_fips_response = requests.get(state_fips_url, verify=certifi.where())
state_fips = pd.read_csv(io.StringIO(state_fips_response.text), sep="|", usecols=["STATE", "STUSAB"])
state_fips["STATE"] = state_fips["STATE"].astype(str).str.zfill(2)

# Merge Data
df = df.merge(state_fips, left_on="State Code", right_on="STATE", how="left").drop(columns=["STATE"])
df.rename(columns={"STUSAB": "State"}, inplace=True)

# Save to CSV for Tableau
df.to_csv("housing_costs.csv", index=False)

print("Data saved to housing_costs.csv")

Data saved to housing_costs.csv


In [2]:
pip install certifi



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'ipywidgets'